# Proyecto de Telecomunicaciones: Identificar operadores ineficaces

El servicio de telefonía virtual CallMeMaybe está desarrollando una nueva función que brindará a los supervisores y las supervisores información sobre los operadores menos eficaces. Se considera que un operador es ineficaz si tiene una gran cantidad de llamadas entrantes perdidas (internas y externas) y un tiempo de espera prolongado para las llamadas entrantes. Además, si se supone que un operador debe realizar llamadas salientes, un número reducido de ellas también será un signo de ineficacia.

# Fases del proyecto

- Objetivo de proyecto: Identificar a los operadores más ineficaces para poder realizar acciones especificas en ese personal y poder mejorar el rendimiento del servicio.

1. Descarga de datos y estudio de la información general
- En esta fase se descargaran los datos a ocupar y se vera una muestra de ellos, junto con información general (tipos de datos, columnas, etc) para identificar ajustes a realizar en la siguiente fase
2. Preprocesamiento de datos
- En esta fase se realizaran los ajustes previamente identificados y se prepara los datos para el análisis.
3. Análisis Exploratorio de los datos (EDA)
- Describe el comportamiento de los operadores. Identifica cuantos operadores están activos, su número de llamadas tanto salientes como entrantes, número total de llamadas y llamadas perdidas, calcula el tiempo de espera y tiempo de duración de llamadas promedio.
- Describe el comportamiento de los clientes. Identifica que clientes ocupan más el servicio, su tarifa actual, que tanto tiempo llevan con el servicio.
4. Análisis
- Analiza e identifica a los 5 operadores que menos eficacia tienen respecto al resto.
- Analiza e identifica a los 5 operadores que más eficacia tienen respecto al resto
5. Prueba las hipótesis
- Prueba que el número promedio de llamadas entrantes perdidas de los operadores identificados como menos eficaces y de los operadores identificados como más eficaces difiere.
- Prueba que el tiempo de espera promedio de los operadores identificados como menos eficaces y de los operadores identificados como más eficaces difiere.
- Prueba que el número promedio de llamadas salientes de los operadores identificados como menos eficaces y de los operadores identificados como más eficaces difiere.
6. Conclusiones generales, recomendaciones finales y posibles siguientes pasos.
- Comenta las conclusiones y recomendaciones finales de todo el proceso.
- Escribe los posibles next steps basados en tus conclusiones y recomendaciones finales

## Tabla de contenidos <a id='back'></a>

- [Descarga de datos y estudio de la información general](#descarga_datos)
- [Preprocesamiento de datos](#preprocesamiento)
- [Análisis exploratorio de datos (EDA)](#eda)
- [Análisis de los top y low 5](#analisis)
- [Prueba las hipótesis](#hipotesis)
- [Conclusiones generales, recomendaciones finales y posibles siguientes pasos](#conclusiones_recomendaciones)

## Descarga de datos y estudio de la información general

En esta fase se descargaran los datos a ocupar y se vera una muestra de ellos, junto con información general (tipos de datos, columnas, etc) para identificar ajustes a realizar en la siguiente fase


In [1]:
# Cargar todas las librerías
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from plotly import graph_objects as go
from scipy import stats as st

Vamos a cargar los datos de los siguientes datasets:

- `telecom_clients_us.csv`
- `telecom_dataset_us.csv`

In [2]:
# Carga de los archivos de datos
data_clients = pd.read_csv('data/telecom_clients_us.csv')
data = pd.read_csv('data/telecom_dataset_us.csv')

# Verificamos que los datos se hayan cargado correctamente
print(data_clients.head())
print('\n')
print(data.head())

   user_id tariff_plan  date_start
0   166713           A  2019-08-15
1   166901           A  2019-08-23
2   168527           A  2019-10-29
3   167097           A  2019-09-01
4   168193           A  2019-10-16


   user_id                       date direction internal  operator_id  \
0   166377  2019-08-04 00:00:00+03:00        in    False          NaN   
1   166377  2019-08-05 00:00:00+03:00       out     True     880022.0   
2   166377  2019-08-05 00:00:00+03:00       out     True     880020.0   
3   166377  2019-08-05 00:00:00+03:00       out     True     880020.0   
4   166377  2019-08-05 00:00:00+03:00       out    False     880022.0   

   is_missed_call  calls_count  call_duration  total_call_duration  
0            True            2              0                    4  
1            True            3              0                    5  
2            True            1              0                    1  
3           False            1             10                   18  
4   

Los datos se han cargado correctamente. Vamos a estudiar los datos que continen nuestros datasets y verificar que los tipos de datos sean correctos para nuestro análisis

In [3]:
# Imprimimos la información general/resumen sobre nuestro primer dataset data_clients
data_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      732 non-null    int64 
 1   tariff_plan  732 non-null    object
 2   date_start   732 non-null    object
dtypes: int64(1), object(2)
memory usage: 17.3+ KB


In [4]:
data_clients.head()

,user_id,tariff_plan,date_start
0,166713,A,2019-08-15
1,166901,A,2019-08-23
2,168527,A,2019-10-29
3,167097,A,2019-09-01
4,168193,A,2019-10-16


Nuestro dataset `data_clients` que son los datos de los clientes del servicio, contiene los siguientes datos:

- `user_id`: ID de usuario/a
- `tariff_plan`: tarifa actual de la clientela
- `date_start`: fecha de registro de la clientela

In [5]:
# Verificamos que no haya datos duplicados en el dataset completo
print('Datos duplicados en el dataset:', data_clients.duplicated().sum())
# Verificamos que no haya datos duplicados en user_id
print('Datos duplicados en el user_id del dataset:', data_clients['user_id'].duplicated().sum())

Datos duplicados en el dataset: 0
Datos duplicados en el user_id del dataset: 0


### Conclusiones del dataset

Al ver la muestra y resumen del dataset, no encontramos duplicados ni ausentes. Sin embargo, es necesario cambiar el tipo de dato de la columna `date_start` a datetime para poder realizar operaciones con fechas.

In [6]:
# Imprimimos la información general/resumen sobre nuestro segundo dataset data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53902 entries, 0 to 53901
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              53902 non-null  int64  
 1   date                 53902 non-null  object 
 2   direction            53902 non-null  object 
 3   internal             53785 non-null  object 
 4   operator_id          45730 non-null  float64
 5   is_missed_call       53902 non-null  bool   
 6   calls_count          53902 non-null  int64  
 7   call_duration        53902 non-null  int64  
 8   total_call_duration  53902 non-null  int64  
dtypes: bool(1), float64(1), int64(4), object(3)
memory usage: 3.3+ MB


In [7]:
data.head()

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration
0,166377,2019-08-04 00:00:00+03:00,in,False,NaN,True,2,0,4
1,166377,2019-08-05 00:00:00+03:00,out,True,880022.0,True,3,0,5
2,166377,2019-08-05 00:00:00+03:00,out,True,880020.0,True,1,0,1
3,166377,2019-08-05 00:00:00+03:00,out,True,880020.0,False,1,10,18
4,166377,2019-08-05 00:00:00+03:00,out,False,880022.0,True,3,0,25


Nuestro dataset `data` que son los datos sobre las llamadas realizadas por los operadores o recibidas por ellos, contiene las siguientes columnas:

- `user_id`: ID de la cuenta de cliente
- `date`: fecha en la que se recuperaron las estadísticas
- `direction`: - "dirección" de llamada (`out` para saliente, `in` para entrante)
- `internal`: si la llamada fue interna (entre los operadores de un cliente o clienta)
- `operator_id`: identificador del operador
- `is_missed_call`: si fue una llamada perdida
- `calls_count`: número de llamadas
- `call_duration`: duración de la llamada (sin incluir el tiempo de espera)
- `total_call_duration`: duración de la llamada (incluido el tiempo de espera)

In [8]:
# Verificamos que no haya datos duplicados en el dataset completo
print('Datos duplicados en el dataset:', data.duplicated().sum())

Datos duplicados en el dataset: 4900


### Conclusiones del dataset

Al ver la muestra y resumen del dataset, encontramos 2 columnas con datos ausentes `internal` y `operator_id`; por la muestra que vimos de los datos si la llamada es dirección entrante el operator_id no es registrado y de ahi los nulos de esta columna, sin embargo, se debe hacer un análisis a fondo para tomar la mejor decision para tratarlos. Junto a esto, se detectaron 4900 datos duplicados, estos también deben investigarse para tomar la mejor decisión de como tratarlos. Finalmente se deben cambiar los tipos de datos de las siguientes columnas `date` a datetime, `internal` a bool y `operator_id` a int.

## Preprocesamiento de datos

En esta fase haremos los ajustes identificados en la fase anterior, los cuales se encuentran ambos datasets.

### Datos Ausentes

Como comentamos anteriormente, se encontraron 2 columnas con datos ausentes

- `internal`
- `operator_id`

Investiguemos las columnas para tomar la mejor decisión de como procesarlos.

In [9]:
# Vamos a checar el porcentaje de valores ausentes en general
porcentaje_nulos = lambda x: x * 100 / data.user_id.size
data.isna().sum().apply(porcentaje_nulos)

user_id                 0.000000
date                    0.000000
direction               0.000000
internal                0.217061
operator_id            15.160847
is_missed_call          0.000000
calls_count             0.000000
call_duration           0.000000
total_call_duration     0.000000
dtype: float64

In [10]:
## Investiguemos la columna internal
# Veamos una muestra de los datos donde hay ausentes
data[data['internal'].isna()].head()

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration
1007,166405,2019-09-18 00:00:00+03:00,in,NaN,NaN,True,1,0,59
1090,166405,2019-10-01 00:00:00+03:00,in,NaN,NaN,True,1,0,1
1864,166406,2019-08-20 00:00:00+03:00,in,NaN,NaN,True,1,0,36
1924,166406,2019-09-02 00:00:00+03:00,in,NaN,879898.0,False,1,2,9
6210,166541,2019-09-26 00:00:00+03:00,in,NaN,908960.0,False,1,393,423


Parece indicar que todas las llamadas internas vacías son llamadas entrantes, verifiquemos.

In [11]:
# Vemos los valores en direction de las rows con valores ausentes en internal
data[data['internal'].isna()]['direction'].value_counts()

in     115
out      2
Name: direction, dtype: int64

Como podemos observar, tenemos 115 llamadas entrantes y 2 salientes; por lo que nuestra hipotesis del inicio no fue cierta. Tomando en cuenta que sin otra posible conexion con las demás columnas no seria posible recuperar los posibles datos faltantes y que los datos ausentes representan un 0.21% de nuestra total data, se toma la decisión de eliminar los datos ausentes de esta columna.

In [12]:
# Eliminamos los datos ausentes de la columna internal
data = data[~data['internal'].isna()]
# Vamos a checar el porcentaje de valores ausentes en general
porcentaje_nulos = lambda x: x * 100 / data.user_id.size
data.isna().sum().apply(porcentaje_nulos)

user_id                 0.00000
date                    0.00000
direction               0.00000
internal                0.00000
operator_id            15.08785
is_missed_call          0.00000
calls_count             0.00000
call_duration           0.00000
total_call_duration     0.00000
dtype: float64

In [13]:
## Investiguemos la columna operator_id
# Veamos una muestra de los datos donde hay ausentes
data[data['operator_id'].isna()].head()

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration
0,166377,2019-08-04 00:00:00+03:00,in,False,NaN,True,2,0,4
7,166377,2019-08-05 00:00:00+03:00,in,False,NaN,True,6,0,35
9,166377,2019-08-06 00:00:00+03:00,in,False,NaN,True,4,0,62
17,166377,2019-08-07 00:00:00+03:00,in,False,NaN,True,2,0,24
27,166377,2019-08-12 00:00:00+03:00,in,False,NaN,True,2,0,34


In [14]:
# Veamos los valores unicos de las columnas direction, internal, is_missed_call y user_id
print('Direction')
print(data[data['operator_id'].isna()]['direction'].value_counts())
print('\nInternal')
print(data[data['operator_id'].isna()]['internal'].value_counts())
print('\nIs_missed_call')
print(data[data['operator_id'].isna()]['is_missed_call'].value_counts())
print('\nuser_id')
print(data[data['operator_id'].isna()]['user_id'].value_counts())

Direction
in     7917
out     198
Name: direction, dtype: int64

Internal
False    7760
True      355
Name: internal, dtype: int64

Is_missed_call
True     7993
False     122
Name: is_missed_call, dtype: int64

user_id
168252    145
166405    120
166582    118
166782    117
166658    111
         ... 
168010      1
167139      1
166507      1
166554      1
168013      1
Name: user_id, Length: 305, dtype: int64


Como podemos ver, las primeras 3 columnas tienen datos ausentes en las 2 opciones por lo que no podriamos generar una relación desde ellas para rellenar estos datos, sin embargo podemos ver que tenemos 305 clientes sin operador_id y si cada cliente tiene un unico operador, podemos generar una relación y recuperar los datos. Primero verifiquemos que cada cliente tiene un solo operador.

In [15]:
# Verificamos si cada cliente tiene un solo operador
data.groupby('user_id')['operator_id'].nunique()

user_id
166377     5
166391     2
166392     3
166399     1
166405    10
          ..
168583     2
168598     1
168601     2
168603     1
168606     1
Name: operator_id, Length: 307, dtype: int64

Como podemos ver en la tabla, hay clientes que tiene hasta 10 operadores por lo que no es posible saber con seguridad que operador cubrio la llamada. Tomando en cuenta todo esto se decide por ultimo eliminarlos del dataset, la perdida seria del 15% de nuestra información, sin embargo, al no poder recuperar los datos del operador no podriamos realizar un análisis correctamente.

In [16]:
# Eliminamos los datos ausentes de la columna internal
data = data[~data['operator_id'].isna()]
# Vamos a checar el porcentaje de valores ausentes en general
porcentaje_nulos = lambda x: x * 100 / data.user_id.size
data.isna().sum().apply(porcentaje_nulos)

user_id                0.0
date                   0.0
direction              0.0
internal               0.0
operator_id            0.0
is_missed_call         0.0
calls_count            0.0
call_duration          0.0
total_call_duration    0.0
dtype: float64

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45670 entries, 1 to 53900
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              45670 non-null  int64  
 1   date                 45670 non-null  object 
 2   direction            45670 non-null  object 
 3   internal             45670 non-null  object 
 4   operator_id          45670 non-null  float64
 5   is_missed_call       45670 non-null  bool   
 6   calls_count          45670 non-null  int64  
 7   call_duration        45670 non-null  int64  
 8   total_call_duration  45670 non-null  int64  
dtypes: bool(1), float64(1), int64(4), object(3)
memory usage: 3.2+ MB


Al eliminar los datos ausentes, pasamos de 53,902 rows a 45,670 rows, un número todavia apto para realizar nuestro análisis.

### Datos duplicados

Se detectaron un total de 4,900 datos duplicados, los cuales deben estudiarse para ver la mejor opción para tratarlos

In [18]:
# Veamos una muestra de los datos donde hay ausentes
data[data.duplicated()]

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration
8,166377,2019-08-05 00:00:00+03:00,out,False,880020.0,True,8,0,50
44,166377,2019-08-14 00:00:00+03:00,out,False,880026.0,False,10,1567,1654
51,166377,2019-08-15 00:00:00+03:00,out,False,880026.0,False,11,1413,1473
62,166377,2019-08-19 00:00:00+03:00,out,False,880026.0,False,14,1519,1598
78,166377,2019-08-22 00:00:00+03:00,out,False,880026.0,True,6,0,55
...,...,...,...,...,...,...,...,...,...
53861,168601,2019-11-20 00:00:00+03:00,out,False,952914.0,True,1,0,2
53869,168601,2019-11-25 00:00:00+03:00,in,False,952914.0,False,7,1229,1282
53874,168601,2019-11-26 00:00:00+03:00,in,False,952914.0,False,4,539,562
53885,168603,2019-11-20 00:00:00+03:00,out,False,959118.0,True,3,0,89


Una vez viendo una muestra podemos ver que los datos que se encontraron duplicados fue en su mayoria porque compartian user_id, operator_id y otras columnas que tienden a repetirse debido a la naturaleza de como estan los datos, sin embargo las fechas difieren para cada columna.

### Cambio de tipo de datos

Como comentamos anteriormente, se identifico 3 columnas que hay que hacer un cambio en el tipo de datos que tienen. Estas son:

- `date_start` a datetime
- `date` a datetime
- `internal` a bool
- `operator_id` a int

In [19]:
## Ajustamos los datos de fechas
data_clients['date_start'] = pd.to_datetime(data_clients.date_start)
data['date'] = pd.to_datetime(pd.to_datetime(data.date).dt.date)

# Ajustamos los demas datos
data['internal'] = data['internal'].astype(bool)
data['operator_id'] = data['operator_id'].apply(int)

# Verificamos los cambios
print(data_clients.info())
print()
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      732 non-null    int64         
 1   tariff_plan  732 non-null    object        
 2   date_start   732 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 17.3+ KB
None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45670 entries, 1 to 53900
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   user_id              45670 non-null  int64         
 1   date                 45670 non-null  datetime64[ns]
 2   direction            45670 non-null  object        
 3   internal             45670 non-null  bool          
 4   operator_id          45670 non-null  int64         
 5   is_missed_call       45670 non-null  bool          
 6   calls_

Con estos ajustes, nuestros datos estan listos para ser analizados.

## Análisis Exploratorio de los datos (EDA)

En esta fase se deben realizar los siguientes pasos:

- Describe el comportamiento de los operadores. Identifica cuantos operadores están activos, su número de llamadas tanto salientes como entrantes, número total de llamadas y llamadas perdidas, calcula el tiempo de espera y tiempo de duración de llamadas promedio.
- Describe el comportamiento de los clientes. Identifica que clientes ocupan más el servicio, su tarifa actual, que tanto tiempo llevan con el servicio.

In [23]:
# Número de operadores activos
print('Tenemos un total de', data.operator_id.nunique(), 'operadores activos')

Tenemos un total de 1092 operadores activos


In [94]:
# Tabla con los datos para cada operador como su número de llamadas (salientes, entrantes, perdidas y totales)
operator_data = data.groupby('operator_id').agg({'calls_count': 'sum', 'is_missed_call': 'sum'}).reset_index()

# Tabla de llamadas entrantes y salientes
operator_direction = data.groupby(['operator_id', 'direction'])['calls_count'].sum().reset_index()

# Juntamos ambas tablas para crear una tabla unificada
operator_data = operator_data.merge(operator_direction.query('direction == "in"')[['operator_id','calls_count']], on='operator_id')
operator_data = operator_data.merge(operator_direction.query('direction == "out"')[['operator_id','calls_count']], on='operator_id')
operator_data.rename(columns={"calls_count_x": "llamadas_totales", "is_missed_call": "llamadas_perdidas", "calls_count_y": "llamadas_entrantes", "calls_count": "llamadas_salientes"}, inplace=True)
operator_data

,operator_id,llamadas_totales,llamadas_perdidas,llamadas_entrantes,llamadas_salientes
0,879896,1131,50,60,1071
1,879898,7973,100,117,7856
2,880020,54,7,8,46
3,880022,219,33,8,211
4,880026,2439,94,25,2414
...,...,...,...,...,...
539,970244,4,1,2,2
540,970252,8,1,1,7
541,970254,12,2,3,9
542,972412,61,2,1,60


In [95]:
# Agregamos al dataset original el tiempo de espera
data['call_wait'] = data.total_call_duration - data.call_duration

# Tabla de duracion de llamadas y tiempo de espera promedio
operator_time_data = data.groupby('operator_id').agg({'call_duration': 'mean', 'call_wait': 'mean', 'total_call_duration': 'mean'}).reset_index()

# Unimos nuestras 2 tablas para hacer una tabla unificada
operator_data = operator_data.merge(operator_time_data, on='operator_id').sort_values(by='llamadas_totales', ascending=False)
operator_data

,operator_id,llamadas_totales,llamadas_perdidas,llamadas_entrantes,llamadas_salientes,call_duration,call_wait,total_call_duration
17,885876,66049,135,1152,64897,14620.545455,4816.241379,19436.786834
18,885890,66016,110,1363,64653,13238.747292,5727.133574,18965.880866
293,925922,22210,33,456,21754,11759.924051,5907.443038,17667.367089
1,879898,7973,100,117,7856,1115.504000,451.860000,1567.364000
61,893804,6570,130,2632,3938,944.867692,269.089231,1213.956923
...,...,...,...,...,...,...,...,...
456,946072,2,1,1,1,87.500000,3.500000,91.000000
538,970240,2,1,1,1,10.000000,26.500000,36.500000
235,918988,2,1,1,1,8.000000,6.500000,14.500000
494,954318,2,1,1,1,6.000000,5.000000,11.000000


In [62]:
# Numero de clientes con el servicio
print('Tenemos un total de', data_clients.user_id.nunique(), 'clientes activos')

Tenemos un total de 732 clientes activos


In [73]:
# Numero de clientes con los diferentes planes
data_clients.groupby('tariff_plan')['user_id'].nunique().reset_index()

,tariff_plan,user_id
0,A,76
1,B,261
2,C,395


In [86]:
# Creamos la tabla con las llamadas realizadas por el usuario o hacia el usuario y su fecha ultima de llamada
clients_data_calls = data.query('internal == False').groupby('user_id').agg({'calls_count': 'sum', 'date': 'max'}).sort_values(by='calls_count', ascending=False).reset_index()

# Creamos una tabla unica con el plan tarifario y la fecha de inicio
clients_data_calls = clients_data_calls.merge(data_clients, on='user_id')
# Renombramos columnas
clients_data_calls.rename(columns={'calls_count': 'llamadas_totales_cliente', 'date': 'fecha_final', 'date_start' : 'fecha_inicio'}, inplace=True)

# Creamos la columna de dia de antiguedad
clients_data_calls['dias_antiguedad'] = clients_data_calls.fecha_final - clients_data_calls.fecha_inicio
clients_data_calls

,user_id,llamadas_totales_cliente,fecha_final,tariff_plan,fecha_inicio,dias_antiguedad
0,166582,154272,2019-11-28,A,2019-08-09,111 days
1,167626,77247,2019-10-20,A,2019-09-24,26 days
2,168361,71405,2019-11-28,A,2019-10-23,36 days
3,167827,42048,2019-11-28,C,2019-10-02,57 days
4,168062,26744,2019-11-28,A,2019-10-11,48 days
...,...,...,...,...,...,...
285,167364,1,2019-11-14,C,2019-09-13,62 days
286,166481,1,2019-08-09,A,2019-08-05,4 days
287,167139,1,2019-11-08,C,2019-09-03,66 days
288,166548,1,2019-11-09,B,2019-08-08,93 days


### Conclusiones intermedias

- Tenemos un total de 1,092 operadores para un total de 732 clientes
- De estos 732 clientes, 395 están con el plan C, 261 con el plan B y solo 76 con el plan A
- El cliente que mayor número de llamadas tiene tiene 111 dias de antiguiedad, es del plan A y ha realizado 154,272 llamadas
- El operador que mayor número de llamadas tiene es el 885876 y tiene un total de 66,049 llamadas de las cuales solo ha perdido 135

## Análisis

Para esta fase, se deben realizar los siguientes pasos:

- Analiza e identifica a los 5 operadores que menos eficacia tienen respecto al resto.
- Analiza e identifica a los 5 operadores que más eficacia tienen respecto al resto

### Top 5 operadores con mayor eficacia

Son los operadores con menor cantidad de llamadas perdidas y un tiempo pequeño de tiempo de espera para las llamadas entrantes. Aunado a esto, son los que mayor número de llamadas salientes tengan

In [156]:
operator_data['% llamadas perdidas'] = (operator_data['llamadas_perdidas'] / operator_data['llamadas_totales']) * 100
operator_data['% llamadas salientes'] = (operator_data['llamadas_salientes'] / operator_data['llamadas_totales']) * 100
operator_data['% tiempo de espera'] = (operator_data['call_wait'] / operator_data['total_call_duration']) * 100
operator_data

,operator_id,llamadas_totales,llamadas_perdidas,llamadas_entrantes,llamadas_salientes,call_duration,call_wait,total_call_duration,% llamadas perdidas,% llamadas salientes,% tiempo de espera
17,885876,66049,135,1152,64897,14620.545455,4816.241379,19436.786834,0.204394,98.255840,24.779000
18,885890,66016,110,1363,64653,13238.747292,5727.133574,18965.880866,0.166626,97.935349,30.197034
293,925922,22210,33,456,21754,11759.924051,5907.443038,17667.367089,0.148582,97.946871,33.437031
1,879898,7973,100,117,7856,1115.504000,451.860000,1567.364000,1.254233,98.532547,28.829296
61,893804,6570,130,2632,3938,944.867692,269.089231,1213.956923,1.978691,59.939117,22.166292
...,...,...,...,...,...,...,...,...,...,...,...
456,946072,2,1,1,1,87.500000,3.500000,91.000000,50.000000,50.000000,3.846154
538,970240,2,1,1,1,10.000000,26.500000,36.500000,50.000000,50.000000,72.602740
235,918988,2,1,1,1,8.000000,6.500000,14.500000,50.000000,50.000000,44.827586
494,954318,2,1,1,1,6.000000,5.000000,11.000000,50.000000,50.000000,45.454545


In [157]:
# Operadores con menor cantidad de llamadas perdidas
top_menos_perdidas = operator_data.query('llamadas_totales > 1000').sort_values(by='% llamadas perdidas').head(50)

# Operadores con menor cantidad de llamadas perdidas
top_menos_espera = operator_data.query('llamadas_totales > 1000').sort_values(by='% tiempo de espera').head(50)

# Operadores con mayor cantidad de llamadas salientes
top_mas_salientes = operator_data.query('llamadas_totales > 1000').sort_values(by='% llamadas salientes', ascending=False).head(50)

In [158]:
# Unimos los top 50 de cada tabla para ver coincidencias
top_menos_perdidas_mas_salientes = pd.merge(top_mas_salientes, top_menos_perdidas, on='operator_id')['operator_id']
top_unificado = pd.merge(top_menos_perdidas_mas_salientes, top_menos_espera, on='operator_id')
top_unificado

,operator_id,llamadas_totales,llamadas_perdidas,llamadas_entrantes,llamadas_salientes,call_duration,call_wait,total_call_duration,% llamadas perdidas,% llamadas salientes,% tiempo de espera
0,894656,3416,72,20,3396,844.117284,204.746914,1048.864198,2.107728,99.414520,19.520822
1,926486,2626,50,16,2610,1941.640351,517.631579,2459.271930,1.904037,99.390708,21.048164
2,887282,2452,51,29,2423,1456.720339,299.889831,1756.610169,2.079935,98.817292,17.072076
3,882686,3931,88,837,3094,1689.527426,291.890295,1981.417722,2.238616,78.707708,14.731386
4,937752,2253,32,493,1760,1310.899083,353.990826,1664.889908,1.420328,78.118065,21.262116


### Top 5 operadores con menor eficacia

Son los operadores con mayor cantidad de llamadas perdidas y un tiempo prolongado de tiempo de espera para las llamadas entrantes. Aunado a esto, son los que menor número de llamadas salientes tengan

In [159]:
# Operadores con mayor cantidad de llamadas perdidas
top_mayor_perdidas = operator_data.query('llamadas_totales > 1000').sort_values(by='% llamadas perdidas', ascending=False).head(50)

# Operadores con mayor cantidad de llamadas perdidas
top_mayor_espera = operator_data.query('llamadas_totales > 1000').sort_values(by='% tiempo de espera', ascending=False).head(50)

# Operadores con menor cantidad de llamadas salientes
top_menos_salientes = operator_data.query('llamadas_totales > 1000').sort_values(by='% llamadas salientes', ascending=True).head(50)

In [160]:
# Unimos los top 50 de cada tabla para ver coincidencias
top_mayor_perdidas_mayor_salientes = pd.merge(top_menos_salientes, top_mayor_perdidas, on='operator_id')['operator_id']
low_unificado = pd.merge(top_mayor_perdidas_mayor_salientes, top_mayor_espera, on='operator_id')
low_unificado

,operator_id,llamadas_totales,llamadas_perdidas,llamadas_entrantes,llamadas_salientes,call_duration,call_wait,total_call_duration,% llamadas perdidas,% llamadas salientes,% tiempo de espera
0,921306,1067,43,475,592,464.779412,178.875000,643.654412,4.029991,55.482662,27.790534
1,923526,3053,70,1016,2037,631.033981,213.383495,844.417476,2.292827,66.721258,25.269905
2,937760,1113,32,351,762,471.533333,183.666667,655.200000,2.875112,68.463612,28.032153
3,899082,2372,79,713,1659,426.964286,198.781250,625.745536,3.330523,69.940978,31.767106
4,889410,1568,83,464,1104,306.706107,85.354962,392.061069,5.293367,70.408163,21.770833
5,937966,1130,36,265,865,481.554455,193.188119,674.742574,3.185841,76.548673,28.631381


Con en este análisis tenemos nuestro top operadores con mayor y menor eficacia

## Prueba las hipótesis

Para esta fase realizaremos las siguientes pruebas:

- Prueba que el porcentaje promedio de llamadas entrantes perdidas de los operadores identificados como menos eficaces y de los operadores identificados como más eficaces difiere.
- Prueba que el porcentaje promedio de tiempo de espera de los operadores identificados como menos eficaces y de los operadores identificados como más eficaces difiere.
- Prueba que el porcentaje promedio de llamadas salientes de los operadores identificados como menos eficaces y de los operadores identificados como más eficaces difiere.

### Prueba 1

Para esta prueba, nuestra hipotesis nula es que "el porcentaje promedio de llamadas entrantes perdidas de los operadores identificados como menos eficaces y de los operadores identificados como más eficaces son iguales".

Nuestra hipotesis alternativa es que "el porcentaje promedio de llamadas entrantes perdidas de los operadores identificados como menos eficaces y de los operadores identificados como más eficaces difiere."

Como prueba estadistica realizaremos la prueba de igualdad de las medias de dos poblaciones y nuestro indice de confianza sera del 95%

In [163]:
sample_top = top_unificado['% llamadas perdidas']

sample_low = low_unificado['% llamadas perdidas']

alpha = 0.05

results = st.ttest_ind(sample_top, sample_low)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("Rechazamos la hipótesis nula")
else:
    print("No rechazamos la hipótesis nula")

p-value:  0.011380262682745213
Rechazamos la hipótesis nula


Una vez realizado la prueba, podemos ver que nuestro valor p es bastante bajo a nuestro nivel de confianza (95% = 0.05), esto nos indica que la hipotesis nula es rechazada y las medias de porcentaje de llamadas perdidas tienen diferencias significativas

Con esta información y las observaciones anteriormente expuestas, podemos concluir que los operadores identificados son bastante menos eficaces que nuestro top identificado y podriamos enfocarnos en ellos para aumentar su rendimiento en este KPI

### Prueba 2

Para esta prueba, nuestra hipotesis nula es que "el porcentaje de tiempo de espera promedio de los operadores identificados como menos eficaces y de los operadores identificados como más eficaces son iguales".

Nuestra hipotesis alternativa es que "el porcentaje de tiempo de espera promedio de los operadores identificados como menos eficaces y de los operadores identificados como más eficaces difiere."

Como prueba estadistica realizaremos la prueba de igualdad de las medias de dos poblaciones y nuestro indice de confianza sera del 95%

In [164]:
sample_top = top_unificado['% tiempo de espera']

sample_low = low_unificado['% tiempo de espera']

alpha = 0.05

results = st.ttest_ind(sample_top, sample_low)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("Rechazamos la hipótesis nula")
else:
    print("No rechazamos la hipótesis nula")

p-value:  0.001548352858882087
Rechazamos la hipótesis nula


Una vez realizado la prueba, podemos ver que nuestro valor p es bastante bajo a nuestro nivel de confianza (95% = 0.05), esto nos indica que la hipotesis nula es rechazada y las medias de porcentaje de tiempo de espera tienen diferencias significativas

Con esta información y las observaciones anteriormente expuestas, podemos concluir que los operadores identificados son bastante menos eficaces que nuestro top identificado y podriamos enfocarnos en ellos para aumentar su rendimiento en este KPI

### Prueba 3

Para esta prueba, nuestra hipotesis nula es que "el porcentaje promedio de llamadas salientes de los operadores identificados como menos eficaces y de los operadores identificados como más eficaces son iguales".

Nuestra hipotesis alternativa es que "el porcentaje promedio de llamadas salientes de los operadores identificados como menos eficaces y de los operadores identificados como más eficaces difiere."

Como prueba estadistica realizaremos la prueba de igualdad de las medias de dos poblaciones y nuestro indice de confianza sera del 95%

In [165]:
sample_top = top_unificado['% llamadas salientes']

sample_low = low_unificado['% llamadas salientes']

alpha = 0.05

results = st.ttest_ind(sample_top, sample_low)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("Rechazamos la hipótesis nula")
else:
    print("No rechazamos la hipótesis nula")

p-value:  0.0025772646028303756
Rechazamos la hipótesis nula


Una vez realizado la prueba, podemos ver que nuestro valor p es bastante bajo a nuestro nivel de confianza (95% = 0.05), esto nos indica que la hipotesis nula es rechazada y las medias de porcentaje de llamadas salientes tienen diferencias significativas

Con esta información y las observaciones anteriormente expuestas, podemos concluir que los operadores identificados son bastante menos eficaces que nuestro top identificado y podriamos enfocarnos en ellos para aumentar su rendimiento en este KPI

## Conclusiones generales, recomendaciones finales y posibles siguientes pasos.

Una vez realizadas las pruebas podemos dar las siguientes conclusiones y recomendaciones:

Conclusiones
- En las 3 pruebas se detecto una diferencia estadisticamente significativa por lo que podemos confirmar que tenemos operadores que estan siendo ineficaces y que comparados con los de mejor rendimiento, las diferencias son superiores a la media
- De los operadores más eficaces podemos ver que el porcentaje de llamadas perdidas no supera el 2.3% mientras que el porcentaje de llamadas salientes de casi del 80-85% en promedio, mientras que los operadores menos eficaces tienen un porcentaje de llamadas perdidas mayor a 2% llegando uno al 5% mientras que en el porcentaje de llamadas salientes presentan porcentajes menores al 75% hasta casi el 50%.
- El porcentaje de tiempo de espera de operadores eficaces es inferior al 21.5% respecto al total del tiempo de llamada mientras que el de los operadores menos eficaces es superior al 21% llegando algunos al 31%.

Recomendaciones y Next Steps
- Viendo los resultados, nuestra principal recomendación es generar un cuadro de mando donde mensualmente podamos ver las metricas de los operadores y cuando empiecen a bajar de las medias, hablar con ellos y ver que opciones se pueden tomar para recuperar su productividad
- Un futuro next step podria ser desarollar un modelo que identifique la probabilidad que un operador baje su rendimiento en los proximos meses y con ello, poder generar estrategias enfocados a evitar que esto suceda